In [ ]:
import config, src
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
data = src.load_data(config.PROCESSED_DATA_FILE)
X_train, X_test, y_train, y_test = src.split_data(data, target_column='music_genre')

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=0)
dt_classifier.fit(X_train, y_train)

In [ ]:
src.evaluate_model(dt_classifier, X_test, y_test, classes= config.CLASSES)

In [ ]:
src.save_model(dt_classifier, 'decision_tree', metadata=dt_classifier.get_params())

In [ ]:
data = src.load_data(config.PROCESSED_DATA_FOLDER + 'processed_data_combined_rap_hiphop.csv')

X_train, X_test, y_train, y_test = src.split_data(data, target_column='music_genre')

class_weight = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight = dict(zip(np.unique(y_train), class_weight))

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=0, class_weight=class_weight)
dt_classifier.fit(X_train, y_train)

In [ ]:
src.evaluate_model(dt_classifier, X_test, y_test, classes=config.CLASSES_COMBINED_RAP_HIPHOP)

In [ ]:
data = src.load_data(config.PROCESSED_DATA_FOLDER + 'processed_data_combined_rap_hiphop_no_keys.csv')
X_train, X_test, y_train, y_test = src.split_data(data, target_column='music_genre')

In [ ]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

dt_classifier = DecisionTreeClassifier(random_state=0, class_weight=dict(zip(np.unique(y_train), class_weights)))
dt_classifier.fit(X_train, y_train)

In [ ]:
src.evaluate_model(dt_classifier, X_test, y_test, classes=config.CLASSES_COMBINED_RAP_HIPHOP)